<a href="https://colab.research.google.com/github/Ashwani11000/GoogleColaboratory/blob/main/LeetCode_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LEETCODE SQL").getOrCreate()

Code for extracting the data from the leetcode tables.

In [8]:
data ='''
+----+-------+
| id | score |
+----+-------+
| 1  | 3.50  |
| 2  | 3.65  |
| 3  | 4.00  |
| 4  | 3.85  |
| 5  | 4.00  |
| 6  | 3.65  |
+----+-------+
'''
# Just paste the data here.
final_data = list()
for i in data.strip().split('\n'):
  tmp = tuple()
  if i.startswith('+'):
    continue
  else:
    for j in i.split('|'):
      if j != '':
        tmp = tmp + (j.strip(),)
  final_data.append(tmp)
print(final_data)

# tmpData = [('id', 'score'), ('1', '3.50'), ('2', '3.65'), ('3', '4.00'), ('4', '3.85'), ('5', '4.00'), ('6', '3.65')]
# tmp_df = spark.createDataFrame(tmpData[1:], schema = tmpData[0])
# tmp_df.show()
# tmp_df.createOrReplaceTempView('tmp_view')
# spark.sql('''
# ''').show()

[('id', 'score'), ('1', '3.50'), ('2', '3.65'), ('3', '4.00'), ('4', '3.85'), ('5', '4.00'), ('6', '3.65')]


### 615. Average Salary: Departments VS Company

In [ ]:
# SQL Schema:
# Create table If Not Exists salary (id int, employee_id int, amount int, pay_date date)
# Create table If Not Exists employee (employee_id int, department_id int)

# Given two tables as below, write a query to display the comparison result (higher/lower/same) of the average salary of employees in a department to the company's average salary.

# Table: salary
# | id | employee_id | amount | pay_date   |
# |----|-------------|--------|------------|
# | 1  | 1           | 9000   | 2017-03-31 |
# | 2  | 2           | 6000   | 2017-03-31 |
# | 3  | 3           | 10000  | 2017-03-31 |
# | 4  | 1           | 7000   | 2017-02-28 |
# | 5  | 2           | 6000   | 2017-02-28 |
# | 6  | 3           | 8000   | 2017-02-28 |


# The employee_id column refers to the employee_id in the following table employee.


# | employee_id | department_id |
# |-------------|---------------|
# | 1           | 1             |
# | 2           | 2             |
# | 3           | 2             |


# So for the sample data above, the result is:


# | pay_month | department_id | comparison  |
# |-----------|---------------|-------------|
# | 2017-03   | 1             | higher      |
# | 2017-03   | 2             | lower       |
# | 2017-02   | 1             | same        |
# | 2017-02   | 2             | same        |


# Explanation


# In March, the company's average salary is (9000+6000+10000)/3 = 8333.33...
# The average salary for department '1' is 9000, which is the salary of employee_id '1' since there is only one employee in this department. So the comparison result is 'higher' since 9000 > 8333.33 obviously.
# The average salary of department '2' is (6000 + 10000)/2 = 8000, which is the average of employee_id '2' and '3'. So the comparison result is 'lower' since 8000 < 8333.33.
# With he same formula for the average salary comparison in February, the result is 'same' since both the department '1' and '2' have the same average salary with the company, which is 7000.


# Takeaway:
# PARTITION BY cannot use alias in the same SELECT clause.

In [ ]:
data = [
    ('id', 'employee_id', 'amount', 'pay_date'),
    (1, 1, 9000, '2017/03/31'),
    (2, 2, 6000, '2017/03/31'),
    (3, 3, 10000, '2017/03/31'),
    (4, 1, 7000, '2017/02/28'),
    (5, 2, 6000, '2017/02/28'),
    (6, 3, 8000, '2017/02/28')
]

salarydf = spark.createDataFrame(data[1:], schema = data[0] )

emp_data = [
    ('employee_id', 'department_id'),
    (1, 1),
    (2, 2),
    (3, 2)
]

employeedf = spark.createDataFrame(emp_data[1:], schema = emp_data[0] )

salarydf.show()

employeedf.show()
salarydf.createOrReplaceTempView("salary")
employeedf.createOrReplaceTempView("employee")

+---+-----------+------+----------+
| id|employee_id|amount|  pay_date|
+---+-----------+------+----------+
|  1|          1|  9000|2017/03/31|
|  2|          2|  6000|2017/03/31|
|  3|          3| 10000|2017/03/31|
|  4|          1|  7000|2017/02/28|
|  5|          2|  6000|2017/02/28|
|  6|          3|  8000|2017/02/28|
+---+-----------+------+----------+

+-----------+-------------+
|employee_id|department_id|
+-----------+-------------+
|          1|            1|
|          2|            2|
|          3|            2|
+-----------+-------------+



In [ ]:
spark.sql('''
Select distinct month as pay_month,department_id, if(perDepartment =total_avg,'same',if(perDepartment>total_avg,'higher','lower')) as comparison
from (
SELECT *,LEFT(pay_date,7) AS MONTH ,avg(amount) OVER(PARTITION BY department_id,LEFT(pay_date,7) ) as perDepartment ,avg(amount) OVER(PARTITION BY LEFT(pay_date,7)) as total_avg
from salary
inner JOIN employee
ON salary.employee_id = employee.employee_id
) abc
''').show()

+---------+-------------+----------+
|pay_month|department_id|comparison|
+---------+-------------+----------+
|  2017/03|            2|     lower|
|  2017/02|            2|      same|
|  2017/03|            1|    higher|
|  2017/02|            1|      same|
+---------+-------------+----------+



### 569. Median Employee Salary-Hard

In [ ]:
# Schema:
# Create table If Not Exists Employee (Id int, Company varchar(255), Salary int);

# The Employee table holds all employees. The employee table has three columns: Employee Id, Company Name, and Salary.

# +-----+------------+--------+
# |Id   | Company    | Salary |
# +-----+------------+--------+
# |1    | A          | 2341   |
# |2    | A          | 341    |
# |3    | A          | 15     |
# |4    | A          | 15314  |
# |5    | A          | 451    |
# |6    | A          | 513    |
# |7    | B          | 15     |
# |8    | B          | 13     |
# |9    | B          | 1154   |
# |10   | B          | 1345   |
# |11   | B          | 1221   |
# |12   | B          | 234    |
# |13   | C          | 2345   |
# |14   | C          | 2645   |
# |15   | C          | 2645   |
# |16   | C          | 2652   |
# |17   | C          | 65     |
# +-----+------------+--------+
# Write a SQL query to find the median salary of each company. Bonus points if you can solve it without using any built-in SQL functions.

# +-----+------------+--------+
# |Id   | Company    | Salary |
# +-----+------------+--------+
# |5    | A          | 451    |
# |6    | A          | 513    |
# |12   | B          | 234    |
# |9    | B          | 1154   |
# |14   | C          | 2645   |
# +-----+------------+--------+

In [ ]:
emp_data = [('id', 'company', 'salary'),
('1', 'A', '2341'),
('2', 'A', '341'),
('3', 'A', '15'),
('4', 'A', '15314'),
('5', 'A', '451'),
('6', 'A', '513'),
('7', 'B', '15'),
('8', 'B', '13'),
('9', 'B', '1154'),
('10', 'B', '1345'),
('11', 'B', '1221'),
('12', 'B', '234'),
('13', 'C', '2345'),
('14', 'C', '2645'),
('15', 'C', '2645'),
('16', 'C', '2652'),
('17', 'C', '65')]

emp_df = spark.createDataFrame(data = emp_data[1:], schema = emp_data[0])

emp_df.show()
emp_df.createOrReplaceTempView('employee')

+---+-------+------+
| id|company|salary|
+---+-------+------+
|  1|      A|  2341|
|  2|      A|   341|
|  3|      A|    15|
|  4|      A| 15314|
|  5|      A|   451|
|  6|      A|   513|
|  7|      B|    15|
|  8|      B|    13|
|  9|      B|  1154|
| 10|      B|  1345|
| 11|      B|  1221|
| 12|      B|   234|
| 13|      C|  2345|
| 14|      C|  2645|
| 15|      C|  2645|
| 16|      C|  2652|
| 17|      C|    65|
+---+-------+------+



In [ ]:
spark.sql('''
Select id, company, salary
from (
Select *, if(total%2==0, total/2, (total+1)/2) as opt1, if(total%2==0, (total+2)/2, (total+1)/2) as opt2
from (
Select *, row_number() over(partition by company order by cast(salary as int)) as rnk, count(salary) over(partition by company) as total
from employee) emp1
) emp2
where rnk = opt1 or rnk=opt2
''').show()

+---+-------+------+
| id|company|salary|
+---+-------+------+
|  5|      A|   451|
|  6|      A|   513|
| 12|      B|   234|
|  9|      B|  1154|
| 14|      C|  2645|
+---+-------+------+



### 181. Employees Earning More Than Their Managers-Easy

In [ ]:
# 181. Employees Earning More Than Their Managers

# Table: Employee

# +-------------+---------+
# | Column Name | Type    |
# +-------------+---------+
# | id          | int     |
# | name        | varchar |
# | salary      | int     |
# | managerId   | int     |
# +-------------+---------+
# id is the primary key (column with unique values) for this table.
# Each row of this table indicates the ID of an employee, their name, salary, and the ID of their manager.


# Write a solution to find the employees who earn more than their managers.

# Return the result table in any order.

# The result format is in the following example.



# Example 1:

# Input:
# Employee table:
# +----+-------+--------+-----------+
# | id | name  | salary | managerId |
# +----+-------+--------+-----------+
# | 1  | Joe   | 70000  | 3         |
# | 2  | Henry | 80000  | 4         |
# | 3  | Sam   | 60000  | Null      |
# | 4  | Max   | 90000  | Null      |
# +----+-------+--------+-----------+
# Output:
# +----------+
# | Employee |
# +----------+
# | Joe      |
# +----------+
# Explanation: Joe is the only employee who earns more than his manager.

In [6]:
empData = [('id', 'name', 'salary', 'managerId'), ('1', 'Joe', '70000', '3'), ('2', 'Henry', '80000', '4'), ('3', 'Sam', '60000', 'Null'), ('4', 'Max', '90000', 'Null')]

emp_df = spark.createDataFrame(empData[1:], schema = empData[0])
emp_df.show()
emp_df.createOrReplaceTempView('employee')

+---+-----+------+---------+
| id| name|salary|managerId|
+---+-----+------+---------+
|  1|  Joe| 70000|        3|
|  2|Henry| 80000|        4|
|  3|  Sam| 60000|     Null|
|  4|  Max| 90000|     Null|
+---+-----+------+---------+



In [7]:
spark.sql('''
SELECT EMP.NAME AS EMPLOYEE
FROM EMPLOYEE EMP
INNER JOIN EMPLOYEE MGR
ON EMP.MANAGERID = MGR.ID
AND EMP.SALARY > MGR.SALARY
''').show()

+--------+
|EMPLOYEE|
+--------+
|     Joe|
+--------+



### 178. Rank Scores-Medium

In [ ]:
# 178. Rank Scores

# Table: Scores

# +-------------+---------+
# | Column Name | Type    |
# +-------------+---------+
# | id          | int     |
# | score       | decimal |
# +-------------+---------+
# id is the primary key (column with unique values) for this table.
# Each row of this table contains the score of a game. Score is a floating point value with two decimal places.


# Write a solution to find the rank of the scores. The ranking should be calculated according to the following rules:

# The scores should be ranked from the highest to the lowest.
# If there is a tie between two scores, both should have the same ranking.
# After a tie, the next ranking number should be the next consecutive integer value. In other words, there should be no holes between ranks.
# Return the result table ordered by score in descending order.

# The result format is in the following example.



# Example 1:

# Input:
# Scores table:
# +----+-------+
# | id | score |
# +----+-------+
# | 1  | 3.50  |
# | 2  | 3.65  |
# | 3  | 4.00  |
# | 4  | 3.85  |
# | 5  | 4.00  |
# | 6  | 3.65  |
# +----+-------+
# Output:
# +-------+------+
# | score | rank |
# +-------+------+
# | 4.00  | 1    |
# | 4.00  | 1    |
# | 3.85  | 2    |
# | 3.65  | 3    |
# | 3.65  | 3    |
# | 3.50  | 4    |
# +-------+------+

In [16]:
scoresData = [('id', 'score'), ('1', '3.50'), ('2', '3.65'), ('3', '4.00'), ('4', '3.85'), ('5', '4.00'), ('6', '3.65')]
scores_df = spark.createDataFrame(scoresData[1:], schema = scoresData[0])
scores_df.show()
scores_df.createOrReplaceTempView('scores')

+---+-----+
| id|score|
+---+-----+
|  1| 3.50|
|  2| 3.65|
|  3| 4.00|
|  4| 3.85|
|  5| 4.00|
|  6| 3.65|
+---+-----+



In [23]:
spark.sql('''
select score, dense_rank() over(order by score desc) as rank
from scores
''').show()

spark.sql('''
select sc.score, sc1.rank
from scores sc
inner join
(select s1.id, count(s2.score) as rank
from scores S1
inner join (select distinct score from scores) s2
on s1.score <= s2.score
group by s1.id) sc1
on sc1.id = sc.id
order by sc.score desc
''').show()


+-----+----+
|score|rank|
+-----+----+
| 4.00|   1|
| 4.00|   1|
| 3.85|   2|
| 3.65|   3|
| 3.65|   3|
| 3.50|   4|
+-----+----+

+-----+----+
|score|rank|
+-----+----+
| 4.00|   1|
| 4.00|   1|
| 3.85|   2|
| 3.65|   3|
| 3.65|   3|
| 3.50|   4|
+-----+----+



### FIN